# GABBR1
Pathogenic vartiants in GABBR1 are associated with [Neurodevelopmental disorder with language delay and variable cognitive abnormalities](https://omim.org/entry/620502) (NEDLC). 

In [1]:
from pyphetools.creation import TemplateImporter
from pyphetools.visualization import IndividualTable, QcVisualizer
from IPython.display import display, HTML
template = "input/GABBR1_individuals.xlsx"
hp_json = "../hp.json"
created_by = "0000-0002-0736-9199"

In [2]:
timporter = TemplateImporter(template=template, hp_json=hp_json, created_by=created_by)

In [3]:
individuals, cvalidator = timporter.import_phenopackets_from_template()

HPO version 2024-02-27
Created encoders for 35 fields
 we got OMIM:620502  NM_001470.4
We output 4 GA4GH phenopackets to the directory phenopackets


In [4]:
### Here, we begin QC and display
qc = QcVisualizer(cohort_validator=cvalidator)
display(HTML(qc.to_summary_html()))

In [5]:
# Cediel ML, et al GABBR1 monoallelic de novo variants linked to neurodevelopmental delay and epilepsy. 
# Am J Hum Genet. 2022 Oct 6;109(10):1885-1893.  PMID: 36103875.
PMID = "PMID:36103875"
df = timporter.create_hpoa_from_phenopackets(pmid=PMID, moi="Autosomal dominant")

[pyphetools] Ingested 4 GA4GH phenopackets.
	Neurodevelopmental disorder with language delay and variable cognitive abnormalities (OMIM:620502): n=4
We found a total of 19 unique HPO terms
Extracted disease: Neurodevelopmental disorder with language delay and variable cognitive abnormalities (OMIM:620502)
Wrote HPOA disease file to OMIM-620502.tab


In [6]:
encoder.output_individuals_as_phenopackets(individuals)

NameError: name 'encoder' is not defined

# HPOA file

In [ ]:
table = IndividualTable(individuals)
display(HTML(table.to_html()))